In [1]:
import pandas as pd

from openenergyid.pandera_poc import analyse

In [2]:
# Input data is pure python

df = pd.DataFrame(
    {
        "column1": [1, 4, 0, 10, 9],
        "column2": [-1.3, -1.4, -2.9, -10.1, -20.4],
        "column3": ["value_1", "value_2", "value_3", "value_2", "value_1"],
    }
)

df

,column1,column2,column3
0,1,-1.3,value_1
1,4,-1.4,value_2
2,0,-2.9,value_3
3,10,-10.1,value_2
4,9,-20.4,value_1


In [3]:
output = analyse(df)

In [4]:
output

,column1,column2,column3
0,1,-1.3,value_1
1,4,-1.4,value_2
2,0,-2.9,value_3
3,10,-10.1,value_2
4,9,-20.4,value_1


In [5]:
# Analysis with empty dataframe

analyse(pd.DataFrame())

SchemaError: column 'column1' not in dataframe. Columns in dataframe: []